# Clean all_data.csv

In [15]:
import pandas as pd

df = pd.read_csv('all_data/all_data_manual_copy.csv')
df

,FORMULA,Process Method,Process Condition,Measurement Method,Measuremet T (Â°C),Measuremet P (MPa),Structure: Exp Density (g/cm^3),Structure: Calculated Density (g/cm^3),Structure: Grain Size (Î¼m),Structure: BCC/FCC/other,...,Electrochemistry: Corrosion Rate (mm/year),Electrochemistry: Corrosion Resistance Index,Electrochemistry: Corrosion Current Density (A/mÂ²),Others: DOI,Others: Reference,Others: Details,appendix_1,appendix_2(hardness+-deviation),appendix_3(Exp. Young modulus+-deviation),Unnamed: 82
0,(CuMnNi)75Zn25,NaN,NaN,C,25.0,NaN,8.3,8.255308,NaN,FCC,...,NaN,NaN,NaN,https://doi.org/10.1016/j.dib.2018.11.111,Data in Brief 21 (2018) 2664¡V2678,FCC,NaN,147,0,NaN
1,(CuMnNi)80Zn20,NaN,NaN,C,25.0,NaN,8.3,8.291761,NaN,FCC,...,NaN,NaN,NaN,https://doi.org/10.1016/j.dib.2018.11.111,Data in Brief 21 (2018) 2664¡V2678,FCC,NaN,109,0,NaN
2,(CuMnNi)90Al10,NaN,NaN,C,25.0,NaN,8.1,8.130251,NaN,other,...,NaN,NaN,NaN,https://doi.org/10.1016/j.dib.2018.11.111,Data in Brief 21 (2018) 2664¡V2678,FCC + Im,NaN,241,NaN,NaN
3,(CuMnNi)90Sn10,NaN,NaN,C,25.0,NaN,8.3,8.327535,NaN,other,...,NaN,NaN,NaN,https://doi.org/10.1016/j.dib.2018.11.111,Data in Brief 21 (2018) 2664¡V2678,FCC + Im,NaN,318,NaN,NaN
4,(CuMnNi)95Al5,NaN,NaN,C,25.0,NaN,8.3,8.277480,NaN,FCC,...,NaN,NaN,NaN,https://doi.org/10.1016/j.dib.2018.11.111,Data in Brief 21 (2018) 2664¡V2678,FCC,NaN,166,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3165,Ta1Ti1Zr1,CAST,NaN,T,25.0,NaN,NaN,9.000000,NaN,other,...,NaN,NaN,NaN,10.3390/e21020114,NaN,NaN,NaN,485,157,NaN
3166,Ti0.25NbMoTaW,NaN,NaN,C,25.0,NaN,NaN,13.095647,NaN,BCC,...,NaN,NaN,NaN,https://doi.org/10.1016/j.dib.2018.11.111,Data in Brief 21 (2018) 2664¡V2678,BCC,NaN,478.3,NaN,NaN
3167,Ti0.5NbMoTaW,NaN,NaN,C,25.0,NaN,NaN,12.596898,NaN,BCC,...,NaN,NaN,NaN,https://doi.org/10.1016/j.dib.2018.11.111,Data in Brief 21 (2018) 2664¡V2678,BCC,NaN,481.2,NaN,NaN
3168,Ti0.75NbMoTaW,NaN,NaN,C,25.0,NaN,NaN,12.152845,NaN,BCC,...,NaN,NaN,NaN,https://doi.org/10.1016/j.dib.2018.11.111,Data in Brief 21 (2018) 2664¡V2678,BCC,NaN,495.6,NaN,NaN


# 選擇欄位

In [16]:
selected_columns = [
    'FORMULA', 
    'Process Method', 
    'Measurement Method',
    'Measuremet T (Â°C)',
    'Structure: Exp Density (g/cm^3)', 
    'Structure: Calculated Density (g/cm^3)', 
    # 'Structure: Grain Size (Î¼m)', 
    'Structure: Microstructure',
    'Mechanics: Hardness (HV)', 
    'Mechanics: Yield strength (MPa)', 
    'Mechanics: Ultra tensle strength (MPa)',
    'Mechanics: Exp Young modulus (GPa)', 
    'Mechanics: Calculated Young modulus (GPa)',
    'Mechanics: Fracture Elongation (%)', 
    'Mechanics: Plastic Elongation (%)',
    'Others: DOI', 
    'Others: Reference'
]

print(len(selected_columns))
df_selected = df[selected_columns]
df_selected.to_csv('all_data/all_data_selected.csv', index=False)

16


# 處理 "FORMULA"

In [17]:
# ============================= Process "FORMULA" =============================
# ========================== 指定檢查欄位，指定存在字串，刪除該列 ==========================
import pandas as pd
import re

df = pd.read_csv("all_data/all_data_selected.csv", encoding="utf-8")  # 視情況改編碼

col = "FORMULA"

# 想要刪除的關鍵字（只要包含其中任一個就刪掉該列）
bad_list = ["(", ")"]

# 組成安全的 regex pattern，例如 "字串1|字串2|字串3"
pattern = "|".join(re.escape(s) for s in bad_list)

# 刪除：只保留「不包含這些字串」的列
df = df[~df[col].astype(str).str.contains(pattern, na=False)]
# # 欄位內容 完全等於任一字串 才刪除整列（不用包含）
# df = df[~df[col].astype(str).isin(bad_list)]

# ========================== 刪除只有一個元素的 ==========================
# --- 解析化學式 ---
token_re = re.compile(r'([A-Z][a-z]?)(?:\s*([-+]?\d*\.?\d+(?:[eE][-+]?\d+)?))?')

def parse_formula(formula: str):
    parts = {}
    for el, amt in token_re.findall(str(formula)):
        val = float(amt) if amt else 1.0
        parts[el] = parts.get(el, 0.0) + val
    return parts

def count_elements(formula: str) -> int:
    return len(parse_formula(formula))

# --- 計算元素數量 ---
df["_element_count"] = df[col].map(count_elements)

# --- 篩選 ---
mask_removed = df["_element_count"] == 1
df_removed = df[mask_removed].copy()     # 被刪掉的 (只有一種元素)
df_cleaned = df[~mask_removed].copy()   # 保留下來的 (至少兩種元素)
df_cleaned.drop(columns=["_element_count"], inplace=True)
# --- 統計與輸出 ---
print("=== 被刪掉的列（只有一種元素） ===")
print(df_removed[col])

print(f"\n總共刪掉 {len(df_removed)} 筆，剩下 {len(df_cleaned)} 筆")

# ========================== 補係數1 ==========================
df_cleaned[col] = (
    df_cleaned[col]
      .astype(str)
      .str.replace(r"\s+", "", regex=True)
      .str.replace(r"([A-Z][a-z]?)(?=[A-Z]|$)", r"\g<1>1", regex=True)
)

df_cleaned.to_csv("all_data/all_data_cleaned_0.csv", index=False)




=== 被刪掉的列（只有一種元素） ===
3154    Ni
Name: FORMULA, dtype: object

總共刪掉 1 筆，剩下 3143 筆


# 處理 "Process Method"

In [19]:
# ============================= Process "Process Method" =============================
# ========================== 有包含才保留 ==========================
import pandas as pd

df = pd.read_csv("all_data/all_data_cleaned_0.csv", encoding="utf-8")

col = "Process Method"

# ✅ 想保留空的：targets = ["字串1", "字串2", ""]
# ❌ 不保留空的：targets = ["字串1", "字串2"]
targets = ["CAST", "WROUGHT", "POWDER", "ANNEAL", "Anneal", ""]

# 把非空字串拿出來，給「包含」邏輯用
non_empty_targets = [t for t in targets if t != ""]

# ---------- 第一步 ----------
# 保留條件：
# 1. 如果有設定 "" 在 targets，則空值 / 全空白字串保留
# 2. 或欄位包含任一 non_empty_targets 中的字串
# 其餘刪除

def keep_row(val):
    # 空值處理
    if pd.isna(val) or str(val).strip() == "":
        return "" in targets  # 有把 "" 放進 targets 才保留
    s = str(val)
    return any(t in s for t in non_empty_targets)

mask_keep = df[col].apply(keep_row)
df = df[mask_keep].copy()

# ---------- 第二步 ----------
# 對於保留下來的列：
# - 若欄位值「包含」某個 target 但不等於任何 target，則改成對應的 target
# - 若同時包含多個，依 targets 順序取最前面的
# - 空值只有在 "" 在 targets 時會存在，直接保留為空

def normalize(val):
    if pd.isna(val) or str(val).strip() == "":
        return val  # 是否允許已在第一步決定
    s = str(val)
    for t in targets:
        if t == "":
            continue
        if t in s:
            # 不等於任何 target 才統一
            if s != t:
                return t
            else:
                return s
    return s  # 正常情況下不會進來，因為第一步已篩掉不含的

df[col] = df[col].apply(normalize)

# ========================== 替換字 (有需要再反開) ==========================
replace_map = {
    "Anneal": "ANNEAL"
} 
# 只對完全等於 key 的內容做替換（不影響其他字串）
df[col] = df[col].replace(replace_map)
# =========================================================================

df.to_csv("all_data/all_data_cleaned_1.csv", index=False)
# 或 df.to_excel("output.xlsx", index=False)


# 處理 "Structure: Microstructure"

In [1]:
# ============================= Process "Structure: Microstructure" =============================
# ========================== 完全一樣才保留 ==========================
import pandas as pd
import numpy as np

# 讀取檔案
df = pd.read_csv("all_data/all_data_cleaned_1.csv", encoding="utf-8")   # 或 read_excel

# ===== 設定區 =====
col = "Structure: Microstructure"   # 要檢查的欄位

# 多個允許值：
# - 想保留空值：把 "" 加進去
# - 不想保留空值：不要放 ""
targets = ["FCC", "BCC", "Other", ""]   # 自行調整
# ==================

# 分開處理非空 target
non_empty_targets = [t for t in targets if t != ""]

def keep_cell(val):
    # 空值（NaN 或 全是空白）
    if pd.isna(val) or str(val).strip() == "":
        return "" in targets          # 只有在 targets 有 "" 才保留
    s = str(val)
    # 僅保留「完全等於」任一非空 target 的列
    return s in non_empty_targets

# 建立保留列的遮罩
mask_keep = df[col].apply(keep_cell)

# 套用過濾
df_filtered = df[mask_keep].copy()

# 輸出
df_filtered.to_csv("all_data/all_data_cleaned_Micro.csv", index=False)
# 或 df_filtered.to_excel("output.xlsx", index=False)


# 變更欄位名稱 + FORMULA 補空格

In [2]:
import re
import pandas as pd

df = pd.read_csv("all_data/all_data_cleaned_Micro.csv", encoding="utf-8")

# ========================== 變更欄位名稱 ==========================
col_rename = {
    'Process Method': 'PROPERTY: Processing method',
    'Measurement Method': 'PROPERTY: Type of test',
    'Measuremet T (Â°C)': 'PROPERTY: Test temperature ($^\circ$C)',
    'Structure: Exp Density (g/cm^3)': 'PROPERTY: Exp. Density (g/cm$^3$)',
    'Structure: Calculated Density (g/cm^3)': 'PROPERTY: Calculated Density (g/cm$^3$)',
    'Structure: Microstructure': 'PROPERTY: Microstructure',
    'Mechanics: Hardness (HV)': 'PROPERTY: HV',
    'Mechanics: Yield strength (MPa)': 'PROPERTY: YS (MPa)',
    'Mechanics: Ultra tensle strength (MPa)': 'PROPERTY: UTS (MPa)',
    'Mechanics: Exp Young modulus (GPa)': 'PROPERTY: Exp. Young modulus (GPa)',
    'Mechanics: Calculated Young modulus (GPa)': 'PROPERTY: Calculated Young modulus (GPa)',
    'Mechanics: Fracture Elongation (%)': 'PROPERTY: Elongation (%)',
    'Mechanics: Plastic Elongation (%)': 'PROPERTY: Elongation plastic (%)'
}


df = df.rename(columns=col_rename)


# ========================== 化學式補空格隔開 ==========================
# 1) 化學式解析 & 標準化（元素字母序、空白分隔）
_token_re = re.compile(r'([A-Z][a-z]*)([0-9.]+)')

def normalize_composition_to_spaced_tokens(formula: str) -> str:
    """
    e.g. "Al1Co0.46Cr0.52Fe0.48Ni0.46" -> "Al1 Co0.46 Cr0.52 Fe0.48 Ni0.46"
    與 pre-training 資料一致的文字表示
    """
    pairs = _token_re.findall(str(formula))
    pairs = sorted(pairs, key=lambda x: x[0])          # 元素字母序
    tokens = [f"{el}{amt}" for el, amt in pairs]
    return " ".join(tokens)

df["FORMULA"] = df["FORMULA"].astype(str).map(normalize_composition_to_spaced_tokens)
df.to_csv("all_data/all_data_cleaned_Micro_final.csv", index=False)

# 選擇 target

In [ ]:
# ============================= 指定要做回歸的 target =============================
# 指定檢查欄位，刪除空列
import pandas as pd

def drop_rows_with_nulls(df: pd.DataFrame, cols_to_check, treat_empty_as_null=True):
    """
    在指定欄位任一為空時，刪除整列。
    - cols_to_check: List[str] 欲檢查的欄位名稱
    - treat_empty_as_null: 是否將空字串/只有空白視為空值
    回傳：df_clean, summary(dict)
    """
    cols_to_check = list(cols_to_check)
    # 檢查欄位存在性
    missing = [c for c in cols_to_check if c not in df.columns]
    if missing:
        raise KeyError(f"找不到欄位：{missing}")

    # 將空字串/只有空白視為空值（僅作用於指定欄位）
    if treat_empty_as_null:
        df = df.copy()
        df[cols_to_check] = df[cols_to_check].replace(r'^\s*$', pd.NA, regex=True)

    before = len(df)
    df_clean = df.dropna(subset=cols_to_check)
    after = len(df_clean)

    summary = {
        "rows_before": before,
        "rows_after": after,
        "rows_dropped": before - after,
        "checked_columns": cols_to_check
    }
    return df_clean, summary

# === 範例 ===
target_column = 'PROPERTY: Calculated Density (g/cm$^3$)'
csv_file = "all_data/all_data_cleaned_final.csv"
save_file = "all_data/all_data_calcDensity.csv"

df = pd.read_csv(csv_file)
df2, info = drop_rows_with_nulls(df, [target_column])
print(info)
df2.to_csv(save_file, index=False)


{'rows_before': 1232, 'rows_after': 1207, 'rows_dropped': 25, 'checked_columns': ['PROPERTY: Calculated Density (g/cm$^3$)']}


# Microstructure 分類 專用

In [11]:
# ============================= 處理 Microstructure 要做分類  =============================

import pandas as pd

# 讀檔
df = pd.read_csv("all_data/all_data_Micro_other.csv")

# 建立對應字典
mapping = {
    "FCC": 0,
    "BCC": 1,
    "Other": 2
}

# 直接在原欄位上取代
df["PROPERTY: Microstructure"] = df["PROPERTY: Microstructure"].replace(mapping)

# 存回去（可以改成你想要的檔名）
df.to_csv("all_data/all_data_Micro_other.csv", index=False)


C:\Users\2503032\AppData\Local\Temp\ipykernel_24152\414931815.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["PROPERTY: Microstructure"] = df["PROPERTY: Microstructure"].replace(mapping)
